## Galaxy Zoo main notebook

### Python files required to run ths notebook: data_loader.py, separate_training_validation.py

In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Training settings
parser = argparse.ArgumentParser(description='Galaxy zoo project')
parser.add_argument('--data', type=str, default='data', metavar='D',
                    help="folder where data is located. train_data.zip and test_data.zip need to be found in the folder")
parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                    help='input batch size for training (default: 64)')
parser.add_argument('--epochs', type=int, default=30, metavar='N',
                    help='number of epochs to train (default: 15)')
parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                    help='learning rate (default: 0.01)')
parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                    help='SGD momentum (default: 0.5)')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')


input_args = ""
args = parser.parse_args(input_args)
torch.manual_seed(args.seed)

In [3]:
### Data Initialization and Loading
from data_loader import initialize_data, loader#, data_transforms # data.py in the same folder
initialize_data(args.data) # extracts the zip files, makes a validation set

data/images_training_rev1.zip


In [4]:
from separate_training_validation import separate

label_ids_training, label_ids_validation, label_values_training, label_values_validation = separate()


In [5]:
crop_size = 256
resolution = 64
batch_size = 64
shuffle = False

train_loader = loader(label_ids_training, label_values_training, label_ids_validation, label_values_validation, crop_size, resolution, batch_size, shuffle)